In [74]:
import json
from pathlib import Path
from pprint import pprint

In [ ]:
def file_path (model):
    filePath=f"/Volumes/Jennie/Reasoning/FinReasoning/output/{model}_output.json"
    return filePath

In [57]:
def evaluate(eva_file_path):
    data=json.loads(Path(eva_file_path).read_text())
    a=0
    for question in data:
        if question["Model Answer"] == question["Answer"]:
            a+=1
        else:
            a+=0
    accuracy=(a/len(data)*100).__round__(2)
    return accuracy

In [ ]:
def evaluate_topics(eva_file_path):
    data=pd.read_json(eva_file_path)
    # 计算每个答案是否正确
    data["isCorrect"] = data.apply(lambda row: 1 if row['Model Answer'] == row['Answer'] else 0, axis=1)

    # 计算每个 General Topics 的正确答案数量
    correct_by_topic = data.groupby('General Topics')['isCorrect'].sum().reset_index()

    # 计算每个 General Topics 的总答案数量
    count_by_topic = data.groupby('General Topics')['isCorrect'].count().reset_index()

    # 合并两个 DataFrame
    accuracy_by_topic = pd.merge(correct_by_topic, count_by_topic, on='General Topics')

    # 计算正确率
    accuracy_by_topic['accuracy%'] = round(accuracy_by_topic['isCorrect_x'] / accuracy_by_topic['isCorrect_y'] * 100, 2)

    # 重命名列
    accuracy_by_topic.columns = ['General Topics', 'correct', 'total', 'accuracy%']

    # 计算总体正确率
    total_correct = data['isCorrect'].sum()
    total_count = data['isCorrect'].count()
    overall_accuracy = round((total_correct / total_count) * 100, 2)

    # 创建一个新的 DataFrame 来存储总体正确率
    overall_accuracy_df = pd.DataFrame([['Overall', total_correct, total_count, overall_accuracy]], columns=['General Topics', 'correct', 'total', 'accuracy%'])

    # 将总体正确率 DataFrame 与 accuracy_by_topic 进行合并
    final_df = pd.concat([accuracy_by_topic, overall_accuracy_df], ignore_index=True)

    # 打印结果
    accuracy=list(final_df['accuracy%'])
    return accuracy
    

In [66]:
models=[
        "gpt_o1","gpt_o1_rag",
        "Gemini_Pro1","Gemini_Pro1_rag",
        "deepseek","deepseek_rag",
        "llama","llama_rag",
        "gpt_4o","gpt_4o_rag",
        "gemini_1.5_pro","gemini_1.5_pro_rag",
        ]
topics=['Alternative Investments',
 'Asset Allocation',
 'Capital Market Expectations',
 'Corporate Finance',
 'Credit Risk Measurement and Management',
 'Derivatives',
 'Derivatives and Currency Management',
 'Economics',
 'Equity Investments',
 'Equity Portfolio Management',
 'Ethicaland Professional Standards',
 'Financial Markets and Products',
 'Financial Reporting and Analysis',
 'Fixed Income',
 'Fixed-Income Portfolio Management',
 'Foundation of Risk Management',
 'Liquidity and Treasury Risk Measurement',
 'Market Risk Measurement and Management',
 'Operational Risk',
 'Operational Risk and Resiliency',
 'Portfolio Management',
 'Quantitative Methods',
 'Risk Management and Investment Management',
 'Valuation and Risk Models',
 'Overall']

In [80]:
modelAcc=[]
for model in models:
    modelAcc.append(evaluate(file_path(model)))
result={
    'Model': models,
    'Accuracy%': modelAcc
}

result=pd.DataFrame(result)
result

,Model,Accuracy%
0,gpt_o1,67.67
1,gpt_o1_rag,80.41
2,Gemini_Pro1,70.39
3,Gemini_Pro1_rag,78.68
4,deepseek,70.90
5,deepseek_rag,78.75
6,llama,35.20
7,llama_rag,47.95
8,gpt_4o,79.69
9,gpt_4o_rag,85.28


In [81]:
result.sort_values(by='Accuracy%', ascending=False)

,Model,Accuracy%
9,gpt_4o_rag,85.28
11,gemini_1.5_pro_rag,83.13
1,gpt_o1_rag,80.41
8,gpt_4o,79.69
5,deepseek_rag,78.75
3,Gemini_Pro1_rag,78.68
10,gemini_1.5_pro,76.37
4,deepseek,70.90
2,Gemini_Pro1,70.39
0,gpt_o1,67.67


In [82]:

topic_Acc=[]
for model in models:
    topic_Acc.append(evaluate_topics(file_path(model)))
topic_result=pd.DataFrame(topic_Acc,index=models, columns=topics).T
topic_result

,gpt_o1,gpt_o1_rag,Gemini_Pro1,Gemini_Pro1_rag,deepseek,deepseek_rag,llama,llama_rag,gpt_4o,gpt_4o_rag,gemini_1.5_pro,gemini_1.5_pro_rag
Alternative Investments,67.50,77.50,76.25,81.25,73.75,80.00,36.25,44.87,76.25,88.75,78.72,82.98
Asset Allocation,55.56,55.56,66.67,77.78,44.44,55.56,22.22,27.78,72.22,83.33,84.62,87.69
Capital Market Expectations,16.67,50.00,50.00,82.35,66.67,100.00,66.67,66.67,50.00,50.00,59.57,68.09
Corporate Finance,67.71,83.33,65.62,78.12,69.79,80.21,37.50,46.88,75.79,87.37,81.25,87.50
Credit Risk Measurement and Management,71.07,81.76,49.37,62.42,58.49,65.41,49.69,59.87,72.90,81.29,58.33,66.67
Derivatives,65.98,79.38,80.41,83.51,73.20,82.47,26.80,41.24,85.57,89.69,66.67,74.07
Derivatives and Currency Management,58.33,70.83,45.83,66.67,62.50,66.67,25.00,29.17,62.50,79.17,88.64,93.18
Economics,74.68,85.99,81.65,84.81,80.38,88.61,43.04,58.86,89.03,90.32,40.00,53.33
Equity Investments,63.58,77.78,76.54,84.57,76.54,82.10,38.89,49.38,86.42,88.89,75.47,81.48
Equity Portfolio Management,38.46,61.54,69.23,73.08,50.00,69.23,53.85,65.38,61.54,76.92,58.33,63.93
